In [1]:
import warnings
from copy import deepcopy
from typing import Optional, TypedDict, Union

import numpy as np
import pandas as pd
from pyplier import PLIERResults
from glmnet import ElasticNet
from tqdm.auto import trange, tqdm

/home/milo/.cache/pypoetry/virtualenvs/pyplier-Idb-akax-py3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
Z = pd.read_csv("tests/data/solveU/Z.csv.gz", index_col="gene")
Chat = pd.read_csv("tests/data/solveU/Chat.csv.gz", index_col="pathway")
priorMat = pd.read_csv("tests/data/solveU/priorMat.csv.gz", index_col="gene")
penalty_factor = np.loadtxt("tests/data/solveU/penalty_factor.csv.gz")
U = pd.read_csv("tests/data/solveU/U_complete.csv.gz", index_col="pathway").astype(np.float64)

In [3]:
priorMat.columns.name = "pathway"

In [4]:
pathwaySelection = "fast"
glm_alpha = 0.9
maxPath = 10
target_frac = 0.7
L3 = None

In [5]:
from scipy.stats import rankdata

In [6]:
# %%timeit -n10
Ur = Chat @ Z

In [35]:
Ur

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
pathway,,,,,,,,,,,,,,,,,,,,,
IRIS_Bcell-Memory_IgG_IgA,0.001527,-0.000535,-0.000890,-0.001575,0.001507,-0.000331,0.000491,0.001883,-0.000032,-0.002338,...,0.001086,0.001280,-0.001140,-0.002192,-0.000876,-0.000849,0.000699,0.002459,0.002163,-0.002246
IRIS_Bcell-Memory_IgM,-0.000518,-0.000382,0.005360,-0.001191,0.001569,-0.000736,0.000155,-0.002282,-0.002318,-0.000497,...,-0.003170,-0.000950,0.002705,-0.000530,0.000298,0.002581,-0.000588,-0.000673,0.001373,0.002782
IRIS_Bcell-naive,-0.000416,-0.000040,0.005983,0.002014,0.000526,0.000101,0.000974,-0.001867,-0.000718,0.002067,...,0.004904,0.002521,-0.000007,0.001651,0.003352,0.001233,0.000649,0.000946,-0.000640,-0.000559
IRIS_CD4Tcell-N0,-0.000198,-0.000841,0.001545,0.003329,-0.000835,0.010863,-0.000067,0.001729,-0.001623,-0.001753,...,0.003807,-0.002243,-0.000186,0.002633,0.004533,0.001797,0.000916,0.000545,0.002809,0.001658
IRIS_CD4Tcell-Th1-restimulated12hour,0.002448,-0.000974,-0.001688,0.001069,0.000425,0.002591,-0.001728,-0.000271,-0.001254,0.000736,...,-0.000428,-0.001094,0.000979,-0.001159,0.001264,-0.002353,0.005712,0.003872,0.001442,0.002728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PID_IL4_2PATHWAY,0.000998,0.002458,0.003286,-0.000863,0.000621,0.000887,-0.001249,0.001375,-0.000412,0.001620,...,0.001488,0.005716,0.001162,-0.000352,0.000474,-0.000451,0.002379,0.002399,0.002039,0.000139
REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,-0.000145,-0.001361,-0.000168,-0.000408,-0.002534,-0.000720,-0.001825,-0.002562,-0.001555,-0.000529,...,-0.002604,-0.001770,-0.001052,0.000146,-0.002166,-0.001967,-0.000564,-0.001809,-0.002711,-0.001646
PID_BCR_5PATHWAY,0.002387,0.001807,0.002542,0.000722,0.001291,0.001355,0.000261,0.001289,0.001477,0.001918,...,0.000957,0.000919,0.000950,0.000626,0.000663,0.001647,-0.000410,0.001229,0.001431,0.000964


In [8]:
Ur = Ur.rank(axis="index", ascending=False)

In [9]:
Urm = Ur.min(axis=1)

In [10]:
Urm

pathway
IRIS_Bcell-Memory_IgG_IgA                         51.0
IRIS_Bcell-Memory_IgM                              9.0
IRIS_Bcell-naive                                   1.0
IRIS_CD4Tcell-N0                                   1.0
IRIS_CD4Tcell-Th1-restimulated12hour               2.0
                                                 ...  
PID_IL4_2PATHWAY                                   8.0
REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR    324.0
PID_BCR_5PATHWAY                                  79.0
PID_TELOMERASEPATHWAY                             17.0
PID_PI3KPLCTRKPATHWAY                             46.0
Length: 606, dtype: float64

In [11]:
U = pd.DataFrame(np.zeros(shape=(priorMat.shape[1], Z.shape[1])))

In [ ]:
# if L3 is None:
#     lambdas = np.exp(np.arange(start=-4, stop=-12.125, step=-0.125))
#     results = dict()
#     lMat = np.full((len(lambdas), Z.shape[1]), np.nan)

#     for i in trange(Z.shape[1]):
#         if pathwaySelection == "fast":
#             iip = np.where([Ur.iloc[:, i] <= maxPath])[1]
#         else:
#             iip = np.where([Urm <= maxPath])[1]

#         with warnings.catch_warnings():
#             warnings.simplefilter("ignore")
#             gres = ElasticNet(
#                 lambda_path=lambdas,
#                 lower_limits=0,
#                 standardize=False,
#                 fit_intercept=True,
#                 alpha=glm_alpha,
#                 max_features=150,
#             )

#             gres.fit(
#                 y=Z.iloc[:, i].astype(np.float64).values,
#                 X=priorMat.iloc[:, iip].astype(np.float64).values,
#                 relative_penalties=[penalty_factor[_] for _ in iip],
#             )

#         gres.iip = iip
#         lMat[:, i] = np.sum(np.where(gres.coef_path_ > 0, 1, 0), axis=0)
#         results[i] = deepcopy(gres)

#     fracs = np.mean(np.where(lMat > 0, 1, 0), axis=1)
#     iibest = np.where(abs(target_frac - fracs) == abs((target_frac - fracs)).min())[
#         0
#     ][0]

#     # yeah, so this is not very pythonic, but it matches the R code
#     # TODO: replace this with something like our original attempt
#     for i in trange(Z.shape[1]):
#         U.iloc[results[i].iip, i] = results[i].coef_path_[:, iibest]

#     U.index = priorMat.columns
#     U.columns = Z.columns
    
#     return solveUReturnDict(U=U, L3=lambdas[iibest])
# else:
#     # do one fit with a given lambda
#     results = dict()
#     for i in range(Z.shape[1]):
#         if pathwaySelection == "fast":
#             iip = np.where([Ur.iloc[:, i] <= maxPath])[1]
#         else:
#             iip = np.where([Urm <= maxPath])[1]

#         with warnings.catch_warnings():
#             warnings.simplefilter("ignore")
#             gres = ElasticNet(
#                 lambda_path=[L3 * 0.9, L3, L3 * 1.1],
#                 lower_limits=0,
#                 standardize=False,
#                 fit_intercept=True,
#                 alpha=glm_alpha,
#                 max_features=150,
#             )

#             # try:
#             gres.fit(
#                 y=Z.iloc[:, i].astype(np.float64).values,
#                 X=priorMat.iloc[:, iip].astype(np.float64).values,
#                 relative_penalties=[penalty_factor[_] for _ in iip],
#             )

#         results[i] = pd.Series(
#             data=[_[1] for _ in gres.coef_path_], index=Ur.index[iip]
#         )

#     U = (
#         pd.DataFrame(index=priorMat.columns.set_names("pathway"))
#         .merge(
#             pd.DataFrame(
#                 {i: results[i] for i in range(Z.shape[1])},
#             ).rename_axis(index="pathway", axis="index"),
#             on="pathway",
#             how="left",
#         )
#         .fillna(0)
#     )

In [16]:
lambdas = np.exp(np.arange(start=-4, stop=-12.125, step=-0.125))

In [17]:
results = dict()

In [18]:
lMat = np.full((len(lambdas), Z.shape[1]), np.nan)

In [19]:
lMat

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [20]:
Z.shape[1]

30

In [24]:
if pathwaySelection == "fast":
    iip = np.where([Ur.iloc[:, i] <= maxPath])[1]
else:
    iip = np.where([Urm <= maxPath])[1]

In [25]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    gres = ElasticNet(
        lambda_path=lambdas,
        lower_limits=0,
        standardize=False,
        fit_intercept=True,
        alpha=glm_alpha,
        max_features=150,
    )

    gres.fit(
        y=Z.iloc[:, i].astype(np.float64).values,
        X=priorMat.iloc[:, iip].astype(np.float64).values,
        relative_penalties=[penalty_factor[_] for _ in iip],
    )

In [26]:
gres.fit(
    y=Z.iloc[:, i].astype(np.float64).values,
    X=priorMat.iloc[:, iip].astype(np.float64).values,
    relative_penalties=[penalty_factor[_] for _ in iip],
)

ElasticNet(alpha=0.9,
           lambda_path=array([1.83156389e-02, 1.61634946e-02, 1.42642339e-02, 1.25881422e-02,
       1.11089965e-02, 9.80365504e-03, 8.65169520e-03, 7.63509422e-03,
       6.73794700e-03, 5.94621736e-03, 5.24751840e-03, 4.63091873e-03,
       4.08677144e-03, 3.60656314e-03, 3.18278080e-03, 2.80879419e-03,
       2.47875218e-03, 2.18749112e-03, 1.93045414e-03, 1.70361980e-03,
       1.50343919...
       7.48518299e-05, 6.60565080e-05, 5.82946637e-05, 5.14448602e-05,
       4.53999298e-05, 4.00652974e-05, 3.53575009e-05, 3.12028850e-05,
       2.75364493e-05, 2.43008313e-05, 2.14454083e-05, 1.89255064e-05,
       1.67017008e-05, 1.47391992e-05, 1.30072977e-05, 1.14788999e-05,
       1.01300936e-05, 8.93977622e-06, 7.88932483e-06, 6.96230472e-06,
       6.14421235e-06]),
           lower_limits=0, max_features=150, standardize=False)

In [ ]:
for i in trange(Z.shape[1]):
    if pathwaySelection == "fast":
        iip = np.where([Ur.iloc[:, i] <= maxPath])[1]
    else:
        iip = np.where([Urm <= maxPath])[1]

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        gres = ElasticNet(
            lambda_path=lambdas,
            lower_limits=0,
            standardize=False,
            fit_intercept=True,
            alpha=glm_alpha,
            max_features=150,
        )

        gres.fit(
            y=Z.iloc[:, i].astype(np.float64).values,
            X=priorMat.iloc[:, iip].astype(np.float64).values,
            relative_penalties=[penalty_factor[_] for _ in iip],
        )

    gres.iip = iip
    lMat[:, i] = np.sum(np.where(gres.coef_path_ > 0, 1, 0), axis=0)
    results[i] = deepcopy(gres)

In [27]:
gres.iip = iip
lMat[:, i] = np.sum(np.where(gres.coef_path_ > 0, 1, 0), axis=0)
results[i] = deepcopy(gres)

{0: ElasticNet(alpha=0.9,
            lambda_path=array([1.83156389e-02, 1.61634946e-02, 1.42642339e-02, 1.25881422e-02,
        1.11089965e-02, 9.80365504e-03, 8.65169520e-03, 7.63509422e-03,
        6.73794700e-03, 5.94621736e-03, 5.24751840e-03, 4.63091873e-03,
        4.08677144e-03, 3.60656314e-03, 3.18278080e-03, 2.80879419e-03,
        2.47875218e-03, 2.18749112e-03, 1.93045414e-03, 1.70361980e-03,
        1.50343919...
        7.48518299e-05, 6.60565080e-05, 5.82946637e-05, 5.14448602e-05,
        4.53999298e-05, 4.00652974e-05, 3.53575009e-05, 3.12028850e-05,
        2.75364493e-05, 2.43008313e-05, 2.14454083e-05, 1.89255064e-05,
        1.67017008e-05, 1.47391992e-05, 1.30072977e-05, 1.14788999e-05,
        1.01300936e-05, 8.93977622e-06, 7.88932483e-06, 6.96230472e-06,
        6.14421235e-06]),
            lower_limits=0, max_features=150, standardize=False)}

In [21]:
i = 0

In [23]:
Ur.iloc[:,0]

pathway
IRIS_Bcell-Memory_IgG_IgA                        184.0
IRIS_Bcell-Memory_IgM                            429.0
IRIS_Bcell-naive                                 415.0
IRIS_CD4Tcell-N0                                 384.0
IRIS_CD4Tcell-Th1-restimulated12hour              98.0
                                                 ...  
PID_IL4_2PATHWAY                                 254.0
REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR    375.0
PID_BCR_5PATHWAY                                 106.0
PID_TELOMERASEPATHWAY                             17.0
PID_PI3KPLCTRKPATHWAY                            535.0
Name: V1, Length: 606, dtype: float64

In [22]:
iip = np.where([Ur.iloc[:, i] <= maxPath])[1]
iip

array([ 62, 194, 208, 230, 233, 238, 349, 354, 358, 366])

In [29]:
fracs = np.mean(np.where(lMat > 0, 1, 0), axis=1)
iibest = np.where(abs(target_frac - fracs) == abs((target_frac - fracs)).min())[
    0
][0]

In [32]:
# yeah, so this is not very pythonic, but it matches the R code
# TODO: replace this with something like our original attempt
for i in trange(Z.shape[1]):
    U.iloc[results[i].iip, i] = results[i].coef_path_[:, iibest]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 12748.64it/s]


In [33]:
U

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
U.index = priorMat.columns
U.columns = Z.columns

In [37]:
Ur = Chat @ Z  # get U by OLS

Ur = Ur.rank(axis="index", ascending=False)  # rank
Urm = Ur.min(axis=1)

U = pd.DataFrame(np.zeros(shape=(priorMat.shape[1], Z.shape[1])))
if L3 is None:
    lambdas = np.exp(np.arange(start=-4, stop=-12.125, step=-0.125))
    results = dict()
    lMat = np.full((len(lambdas), Z.shape[1]), np.nan)

In [38]:
%%timeit -n 10
for i in trange(Z.shape[1]):
    if pathwaySelection == "fast":
        iip = np.where([Ur.iloc[:, i] <= maxPath])[1]
    else:
        iip = np.where([Urm <= maxPath])[1]

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        gres = ElasticNet(
            lambda_path=lambdas,
            lower_limits=0,
            standardize=False,
            fit_intercept=True,
            alpha=glm_alpha,
            max_features=150,
        )

        gres.fit(
            y=Z.iloc[:, i].astype(np.float64).values,
            X=priorMat.iloc[:, iip].astype(np.float64).values,
            relative_penalties=[penalty_factor[_] for _ in iip],
        )

    gres.iip = iip
    lMat[:, i] = np.sum(np.where(gres.coef_path_ > 0, 1, 0), axis=0)
    results[i] = deepcopy(gres)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:04<00:00,  7.46it/s]

4.1 s ± 89.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
U

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
IRIS_Bcell-Memory_IgG_IgA,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_Bcell-Memory_IgM,0.0,0.0,0.000384,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_Bcell-naive,0.0,0.0,0.001796,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_CD4Tcell-N0,0.0,0.0,0.000000,0.0,0.0,0.002874,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_CD4Tcell-Th1-restimulated12hour,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PID_IL4_2PATHWAY,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PID_BCR_5PATHWAY,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PID_TELOMERASEPATHWAY,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
